In [ ]:
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import time
import os
import copy
from glob import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

import torchvision
from torchvision import datasets, models, transforms

plt.ion()   # interactive mode
import torch
import numpy as np
import random

# Set random seeds
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed()


In [ ]:
# Set up the data folders. This requires the working directory to have the cell count zip file.

import zipfile
import os
import shutil

with zipfile.ZipFile('data/cell-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

for phase in ['train', 'val']:
    for d in ['images', 'labels']:
        if not os.path.isdir(f'./data/{phase}/{d}'):
            os.makedirs(f'./data/{phase}/{d}')

train_idxs = range(1,181) # Idx to use as train\val
val_idxs = range(181,201)


for i in train_idxs:
    full_num = str(i).zfill(3)
    shutil.copy(f'./data/images/{full_num}cell.png', f'./data/train/images/{full_num}cell.png')
    shutil.copy(f'./data/labels/{full_num}dots.png', f'./data/train/labels/{full_num}dots.png')

for i in val_idxs:
    full_num = str(i).zfill(3)
    shutil.copy(f'./data/images/{full_num}cell.png', f'./data/val/images/{full_num}cell.png')
    shutil.copy(f'./data/labels/{full_num}dots.png', f'./data/val/labels/{full_num}dots.png')


In [ ]:
# We tried a bunch of these normalizations but these worked best. Normalize might also help but we didn't have time to test it.
def change_image_range(x):
    return x / 255

data_transforms = {
    'train':
    transforms.Compose([
        change_image_range,
        transforms.GaussianBlur(9),
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomVerticalFlip(0.5),
        # transforms.Normalize(means, stds)
        # transforms.ColorJitter(brightness=),
        # transforms.ElasticTransform(),
        # transforms.RandomRotation([torch.pi, -torch.pi/2, torch.pi/2], interpolation=transforms.InterpolationMode.BILINEAR),
    ]),
    'val':
    transforms.Compose([
        change_image_range,
        # transforms.Normalize(means, stds)
    ])
}

In [ ]:
# Loads the dataset counting on the directory structure outlines above
class CellsDataset(Dataset):
    def __init__(self, root_dir, transform=None, target_transform=None):

        set_seed()
        self.transform = transform
        self.target_transform = target_transform

        self.data_paths = []

        for image_path in glob(root_dir + '/images/*.png'):
            image_name = image_path.split('/')[-1].split('cell')[0]
            label_path = f'{root_dir}/labels/{image_name}dots.png'

            if os.path.exists(f'{root_dir}/labels/{image_name}dots.png'):
                self.data_paths.append((image_path, label_path))
            else:
                print('Missing label for', image_path)


    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()


        img_path, label_path = self.data_paths[idx]

        image = torchvision.io.read_image(img_path)
        label = torchvision.io.read_image(label_path)

        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            label = self.target_transform(label)

        sample = (image, label)

        return sample


In [ ]:
data_dir = r'./data'
# Create a dictionary of train and val datasets from images in folders
image_datasets = {x: CellsDataset(os.path.join(data_dir, x),
                                  data_transforms[x],
                                  target_transform=lambda x: x[0].sum() / 255
                                  )
                  for x in ['train', 'val']}


dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=32,
                                             shuffle=False, num_workers=2),
    'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=32,
                                          shuffle=False, num_workers=2)
  }

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

print('dataset_sizes: ', dataset_sizes)

dataset_sizes:  {'train': 180, 'val': 20}


In [ ]:
# Check for the availability of a GPU, and use CPU otherwise
# If you are using Google Colab, be sure to change your runtime environment to GPU first.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
# A helper function to show an image from a tensor. We need to restore it to the original dynamic range before normalization
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    # mean = np.array(means)
    # std = np.array(stds)
    # inp = std * inp + mean
    # inp = np.clip(inp, 0, 1)
    fig = plt.figure(figsize=(5,3), dpi=300)
    plt.imshow(inp)
    if title is not None:
       plt.title(title, fontsize=5)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25):
    set_seed()
    since = time.time()

    # Init variables that will save info about the best model
    best_model_wts = copy.deepcopy(model.state_dict())
    best_mae = torch.inf

    train_res= np.zeros((2,num_epochs))
    val_res=np.zeros((2,num_epochs))
    dict_res={'train':train_res, 'val':val_res}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                # Set model to training mode.
                model.train()
            else:
                # Set model to evaluate mode. In evaluate mode, we don't perform backprop and don't need to keep the gradients
                model.eval()

            running_loss = 0.0
            running_ae = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                # Prepare the inputs for GPU/CPU
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # ===== forward pass ======
                with torch.set_grad_enabled(phase=='train'):
                    # If we're in train mode, we'll track the gradients to allow back-propagation
                    preds = model(inputs).squeeze() # apply the model to the inputs.
                    loss = criterion(preds, labels)

                    # ==== backward pass + optimizer step ====
                    # This runs only in the training phase
                    if phase == 'train':
                        loss.backward() # Perform a step in the opposite direction of the gradient
                        optimizer.step() # Adapt the optimizer

                # Collect statistics
                running_loss += (loss.item() * inputs.size(0))
                batch_ae = (preds - labels).abs().sum()
                running_ae += batch_ae

            if phase == 'train':
                # Adjust the learning rate based on the scheduler
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_mae = running_ae / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} MAE: {epoch_mae:.4f}')

            dict_res[phase][0,epoch]=epoch_loss
            dict_res[phase][1,epoch]=epoch_mae

            # Keep the results of the best model so far
            if phase == 'val' and epoch_mae < best_mae:
                best_mae = epoch_mae
                # deepcopy the model
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {(time_elapsed // 60):.0f}m {(time_elapsed % 60):.0f}s')
    print(f'Best val MAE: {best_mae:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, dict_res,best_mae


In [ ]:
# Our network is completely made up. We also tried unet with added linear layers but it was hard to train and not as effective.
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        set_seed(42)
        self.stuff = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
                nn.BatchNorm2d(16),
                nn.ReLU(),
                nn.Conv2d(in_channels=16, out_channels=24, kernel_size=3, padding=1),
                nn.BatchNorm2d(24),
                nn.ReLU(),
                nn.Conv2d(in_channels=24, out_channels=32, kernel_size=3, padding=1),
                nn.BatchNorm2d(32),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2),
                nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
                nn.BatchNorm2d(128),
                nn.ReLU(),
                nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
                nn.BatchNorm2d(256),
                nn.ReLU(),
                nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
                nn.BatchNorm2d(512),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2),
                nn.Flatten(),
                nn.Linear(64*64*512, 64),
                nn.ReLU(),
                nn.Linear(64, 1)
            )
    def forward(self, x):
        return self.stuff(x)


In [ ]:
model = Net()
model = model.to(device)
criterion = nn.MSELoss()
optimizer_ft = optim.Adam(model.parameters(), lr=0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=40, gamma=0.1)
num_epochs = 20

model,dict_res,best_mae = train_model(model.to(device),
                        dataloaders,
                      criterion,
                      optimizer_ft,
                      exp_lr_scheduler,
                      num_epochs=num_epochs)

Epoch 0/19
----------
train Loss: 12500.5146 MAE: 88.7813
val Loss: 29248.4785 MAE: 157.9612

Epoch 1/19
----------
train Loss: 1580.4199 MAE: 30.3662
val Loss: 25688.5977 MAE: 146.3291

Epoch 2/19
----------
train Loss: 1500.0739 MAE: 32.9171
val Loss: 13617.5342 MAE: 97.5603

Epoch 3/19
----------
train Loss: 373.8204 MAE: 16.7142
val Loss: 9629.6436 MAE: 77.7396

Epoch 4/19
----------
train Loss: 532.4621 MAE: 17.6654
val Loss: 4168.8267 MAE: 50.9891

Epoch 5/19
----------
train Loss: 151.2976 MAE: 8.9666
val Loss: 4195.9634 MAE: 51.0146

Epoch 6/19
----------
train Loss: 242.4358 MAE: 12.5836
val Loss: 2396.5696 MAE: 39.6073

Epoch 7/19
----------
train Loss: 105.1125 MAE: 8.3944
val Loss: 1972.6467 MAE: 34.9085

Epoch 8/19
----------
train Loss: 164.1655 MAE: 10.2226
val Loss: 743.1985 MAE: 22.2532

Epoch 9/19
----------
train Loss: 69.6490 MAE: 6.4123
val Loss: 460.1715 MAE: 16.6682

Epoch 10/19
----------
train Loss: 101.6540 MAE: 7.8888
val Loss: 115.7043 MAE: 8.4311

Epoch 11/

In [ ]:
# bst_lr = 0
# bst_score = 120493
# for learn in [0.00005,0.0001,0.0005]:
#   model = Net()
#   model = model.to(device)
#   criterion = nn.MSELoss()
#   optimizer_ft = optim.Adam(model.parameters(), lr=learn)  # reduce the learning rate
#   exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=50, gamma=0.1)
#   num_epochs = 50

#   model,dict_res,best_mae = train_model(model.to(device),
#                           dataloaders,
#                         criterion,
#                         optimizer_ft,
#                         exp_lr_scheduler,
#                         num_epochs=num_epochs)
#   if best_mae < bst_score:
#     bst_score = best_mae
#     bst_lr = learn